# CIV1499 - Introduction to Data Science
## Project 2 - Toronto Bike Share

## Setup Notebook

In [1]:
# Import 3rd party libraries
import os
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
import numpy as np
import datetime


# Configure Notebook
%matplotlib inline
plt.style.use('fivethirtyeight')
sns.set_context("notebook")
import warnings
warnings.filterwarnings('ignore')

# 1. Weather Data

In [2]:
# Set weather data path
path_weather = os.path.join(os.path.abspath(os.getcwd()), 'data', 'weather')

# Get weather file names
weather_filenames = [filename for filename in os.listdir(path_weather) if 'climate' in filename]

# Print file names
print(weather_filenames[0:5])

['en_climate_hourly_ON_6158359_01-2017_P1H.csv', 'en_climate_hourly_ON_6158359_01-2018_P1H.csv', 'en_climate_hourly_ON_6158359_01-2019_P1H.csv', 'en_climate_hourly_ON_6158359_01-2020_P1H.csv', 'en_climate_hourly_ON_6158359_01-2021_P1H.csv']


# 2. Bike Share Trip Data

In [3]:
# Set bikeshare data path
path_ridership = os.path.join(os.path.abspath(os.getcwd()), 'data', 'ridership')

# Create a list of file names for bike share trips data
trips_filenames = [filename for filename in os.listdir(os.path.join(path_ridership, 'release')) if 'bike_share' in filename]

# Create a dictionary where key:value pairs correspond to the file name and DataFrame respectively
trips_data = {filename: pd.read_csv(os.path.join(path_ridership, 'release', filename)) for filename in trips_filenames}  

# Now lets print out the column names for the first month of each year
print('2017\n{}\n'.format(trips_data['bike_share_2017-1.csv'].columns.tolist()))
print('2018\n{}\n'.format(trips_data['bike_share_2018-1.csv'].columns.tolist()))
print('2019\n{}\n'.format(trips_data['bike_share_2019-1.csv'].columns.tolist()))
print('2020\n{}\n'.format(trips_data['bike_share_2020-1.csv'].columns.tolist()))
print('2021\n{}\n'.format(trips_data['bike_share_2021-1.csv'].columns.tolist()))

2017
['trip_id', 'trip_start_time', 'trip_stop_time', 'trip_duration_seconds', 'from_station_id', 'from_station_name', 'to_station_id', 'to_station_name', 'user_type']

2018
['trip_id', 'trip_duration_seconds', 'from_station_id', 'trip_start_time', 'from_station_name', 'trip_stop_time', 'to_station_id', 'to_station_name', 'user_type']

2019
['Trip Id', 'Subscription Id', 'Trip  Duration', 'Start Station Id', 'Start Time', 'Start Station Name', 'End Station Id', 'End Time', 'End Station Name', 'Bike Id', 'User Type']

2020
['Trip Id', 'Subscription Id', 'Trip  Duration', 'Start Station Id', 'Start Time', 'Start Station Name', 'End Station Id', 'End Time', 'End Station Name', 'Bike Id', 'User Type']

2021
['Trip Id', 'Trip  Duration', 'Start Station Id', 'Start Time', 'Start Station Name', 'End Station Id', 'End Time', 'End Station Name', 'Bike Id', 'User Type']



# 3. Station Data

In [5]:
# Set stations data path
path_station = os.path.join(os.path.abspath(os.getcwd()), 'data', 'stations')

# Import stations
stations = pd.read_csv(os.path.join(path_station, 'bikeshare_stations.csv'))

# View DataFrame
stations.head()

,station_id,name,lat,lon
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954
1,7001,Wellesley Station Green P,43.664964,-79.383550
2,7002,St. George St / Bloor St W,43.667333,-79.399429
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761
4,7004,University Ave / Elm St,43.656518,-79.389099


# 4. First Step -  Concatenation of CSV files

## 4.1  Weather Data
First, let's take a look at the column names for the first month of each year

In [6]:
# Now lets print out the column names for the first month of each year
print('2017\n{}\n'.format(pd.read_csv(path_weather+"//"+weather_filenames[0],low_memory=False).columns.to_list()))
print('2018\n{}\n'.format(pd.read_csv(path_weather+"//"+weather_filenames[1],low_memory=False).columns.to_list()))
print('2019\n{}\n'.format(pd.read_csv(path_weather+"//"+weather_filenames[2],low_memory=False).columns.to_list()))
print('2020\n{}\n'.format(pd.read_csv(path_weather+"//"+weather_filenames[3],low_memory=False).columns.to_list()))
print('2021\n{}\n'.format(pd.read_csv(path_weather+"//"+weather_filenames[4],low_memory=False).columns.to_list()))


2017
['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID', 'Date/Time', 'Year', 'Month', 'Day', 'Time', 'Temp (°C)', 'Temp Flag', 'Dew Point Temp (°C)', 'Dew Point Temp Flag', 'Rel Hum (%)', 'Rel Hum Flag', 'Wind Dir (10s deg)', 'Wind Dir Flag', 'Wind Spd (km/h)', 'Wind Spd Flag', 'Visibility (km)', 'Visibility Flag', 'Stn Press (kPa)', 'Stn Press Flag', 'Hmdx', 'Hmdx Flag', 'Wind Chill', 'Wind Chill Flag', 'Weather']

2018
['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID', 'Date/Time', 'Year', 'Month', 'Day', 'Time', 'Temp (°C)', 'Temp Flag', 'Dew Point Temp (°C)', 'Dew Point Temp Flag', 'Rel Hum (%)', 'Rel Hum Flag', 'Wind Dir (10s deg)', 'Wind Dir Flag', 'Wind Spd (km/h)', 'Wind Spd Flag', 'Visibility (km)', 'Visibility Flag', 'Stn Press (kPa)', 'Stn Press Flag', 'Hmdx', 'Hmdx Flag', 'Wind Chill', 'Wind Chill Flag', 'Weather']

2019
['Longitude (x)', 'Latitude (y)', 'Station Name', 'Climate ID', 'Date/Time', 'Year', 'Month', 'Day', 'Time', 'Temp (°C)', 'Temp F

As it can be seen, there are issues with the consistency of names of columns like `Date/Time` and `Time`. Also, `2021 dataset` has two additional columns including, `Precip. Amount (mm)`, `Precip. Amount Flag` which are not reported in the other datasets. So, in the following line, we try to concatenate these weather csv files based on the corrected column names, as follows:
- Define a dictionary which its key are the unique column names for the concatenated dataset and the values are column names for each key in different datasets
- Use the dictionary to change the name of each CSV file and 
- then, concatenate those CSV files


In [7]:
#Creating a Dictionary for common columns' names in weather datasets
weather_col_dic ={
    "Date/Time": ["Date/Time", "Date/Time (LST)"],
    "Time": ["Time", "Time (LST)"],
}

#Create a new DF for Weather
df_weather = pd.DataFrame()
for i in range(len(weather_filenames)):
    temp_df = pd.read_csv(path_weather+"//"+weather_filenames[i],low_memory=False)
    for col in temp_df.columns:
        for key,val in weather_col_dic.items():

            for item in val:
                if col == item:
                    temp_df = temp_df.rename(columns={col:key})
    
    
    temp_df['Excel_File'] = str(weather_filenames[i].split('_')[5])
    df_weather = pd.concat([df_weather,temp_df],
                           ignore_index=True, sort = False, axis=0)

    
#Weather Dataframe
df_weather.head()

,Longitude (x),Latitude (y),Station Name,Climate ID,Date/Time,Year,Month,Day,Time,Temp (°C),...,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather,Excel_File,Precip. Amount (mm),Precip. Amount Flag
0,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 00:00,2017,1,1,00:00,1.5,...,99.81,NaN,NaN,NaN,NaN,NaN,NaN,01-2017,NaN,NaN
1,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 01:00,2017,1,1,01:00,1.5,...,100.01,NaN,NaN,NaN,NaN,NaN,NaN,01-2017,NaN,NaN
2,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 02:00,2017,1,1,02:00,1.0,...,100.14,NaN,NaN,NaN,NaN,NaN,NaN,01-2017,NaN,NaN
3,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 03:00,2017,1,1,03:00,1.2,...,100.32,NaN,NaN,NaN,NaN,NaN,NaN,01-2017,NaN,NaN
4,-79.4,43.63,TORONTO CITY CENTRE,6158359,2017-01-01 04:00,2017,1,1,04:00,1.3,...,100.48,NaN,NaN,NaN,NaN,NaN,NaN,01-2017,NaN,NaN


Among all these columns, we are going to use only some of them. Then, it is better to drop some of them. After checking the dataset, it was decided to drop the following columns from the `Weather_df` dataset:
`["Station Name", "Climate ID", "Temp Flag", "Dew Point Temp Flag", "Rel Hum Flag", "Wind Dir Flag", "Wind Spd Flag", "Visibility Flag", "Hmdx Flag", "Wind Chill Flag", "Excel_File", "Precip. Amount (mm)", "Precip. Amount Flag"]`

In [8]:
lst_weather_drop_col = ["Station Name", "Climate ID", "Temp Flag", "Dew Point Temp Flag", "Rel Hum Flag", "Wind Dir Flag",
                        "Wind Spd Flag", "Visibility Flag", "Hmdx Flag", "Wind Chill Flag", "Excel_File", "Precip. Amount (mm)",
                        "Precip. Amount Flag"]
df_weather = df_weather.drop(lst_weather_drop_col,axis = 1)
df_weather.head()

,Longitude (x),Latitude (y),Date/Time,Year,Month,Day,Time,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Stn Press Flag,Hmdx,Wind Chill,Weather
0,-79.4,43.63,2017-01-01 00:00,2017,1,1,00:00,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN,NaN
1,-79.4,43.63,2017-01-01 01:00,2017,1,1,01:00,1.5,-3.9,67.0,27.0,35.0,16.1,100.01,NaN,NaN,NaN,NaN
2,-79.4,43.63,2017-01-01 02:00,2017,1,1,02:00,1.0,-4.3,68.0,26.0,32.0,16.1,100.14,NaN,NaN,NaN,NaN
3,-79.4,43.63,2017-01-01 03:00,2017,1,1,03:00,1.2,-4.3,67.0,26.0,37.0,16.1,100.32,NaN,NaN,NaN,NaN
4,-79.4,43.63,2017-01-01 04:00,2017,1,1,04:00,1.3,-4.4,66.0,26.0,28.0,16.1,100.48,NaN,NaN,NaN,NaN


Since we are going to merge this dataset with the one we would have from the `bikerides`, then we should convert the `Date/Time` column into a `datetime format` and localize the datetime to `EST`. In order to do this, we would use the following codes

In [9]:
df_weather['Date/Time'] = pd.to_datetime(df_weather['Date/Time'])
df_weather['Date/Time'] = df_weather.apply(lambda row: row['Date/Time'].tz_localize(tz ='EST'), axis = 1)

Let's take a look at the weather dataframe.

In [10]:
df_weather.head()

,Longitude (x),Latitude (y),Date/Time,Year,Month,Day,Time,Temp (°C),Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Stn Press Flag,Hmdx,Wind Chill,Weather
0,-79.4,43.63,2017-01-01 00:00:00-05:00,2017,1,1,00:00,1.5,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN,NaN
1,-79.4,43.63,2017-01-01 01:00:00-05:00,2017,1,1,01:00,1.5,-3.9,67.0,27.0,35.0,16.1,100.01,NaN,NaN,NaN,NaN
2,-79.4,43.63,2017-01-01 02:00:00-05:00,2017,1,1,02:00,1.0,-4.3,68.0,26.0,32.0,16.1,100.14,NaN,NaN,NaN,NaN
3,-79.4,43.63,2017-01-01 03:00:00-05:00,2017,1,1,03:00,1.2,-4.3,67.0,26.0,37.0,16.1,100.32,NaN,NaN,NaN,NaN
4,-79.4,43.63,2017-01-01 04:00:00-05:00,2017,1,1,04:00,1.3,-4.4,66.0,26.0,28.0,16.1,100.48,NaN,NaN,NaN,NaN


Let's save the weather dataframe as a CSV file. We will use call dataframe in the second and third step of the project

In [11]:
#Exporting the DataFrame
df_weather.to_csv('df_weather.csv', index=False )

## 4.2  Bike Share Trip Data


Same to the weather datasets, there are some issues with the consistency of names of columns and the number of reported columns in each dataset. So, in the following line, we try to concatenate these Bikeride share csv files based on the corrected column names, as follows:
- Define a dictionary which its key are the unique column names for the concatenated dataset and the values are column names for each key in different datasets
- Use the dictionary to change the name of each CSV file and 
- then, concatenate those CSV files

In [13]:
#Creating a Dictionary of columns' name
bike_col_dic ={
    "ID": ["trip_id", "Trip Id", "ï»¿Trip Id"],
    "Start_Time": ["trip_start_time", "Start Time"],
    "End_Time": ["trip_stop_time","End Time"],
    "Duration": ["trip_duration_seconds","Trip  Duration"],
    "Start_Station_id": ["from_station_id","Start Station Id"],
    "Start_Station_name": ["from_station_name","Start Station Name"],
    "End_Station_id": ["to_station_id","End Station Id"],
    "End_Station_name": ["to_station_name","End Station Name"],
    "User_Type": ["user_type","User Type"]
}
#bike_col_dic


#Create a new DF for BikeRides
bikerides_df = pd.DataFrame()


for i in range(len(trips_filenames)):
    temp_df = pd.read_csv(path_ridership+"\\release\\"+trips_filenames[i], low_memory=False)
    for col in temp_df.columns:
        for key,val in bike_col_dic.items():
            
            for item in val:
                if col == item:
                    temp_df = temp_df.rename(columns={col:key})

    bikerides_df = pd.concat([bikerides_df,temp_df],
                             ignore_index=True, sort = False, axis=0)

# View DataFrame
bikerides_df.head()

,ID,Start_Time,End_Time,Duration,Start_Station_id,Start_Station_name,End_Station_id,End_Station_name,User_Type,Subscription Id,Bike Id
0,712441,2017-01-01 05:03 (UTC),2017-01-01 05:08 (UTC),274,7006.0,Bay St / College St (East Side),7021.0,Bay St / Albert St,Member,NaN,NaN
1,712442,2017-01-01 05:03 (UTC),2017-01-01 05:12 (UTC),538,7046.0,Niagara St / Richmond St W,7147.0,King St W / Fraser Ave,Member,NaN,NaN
2,712443,2017-01-01 05:05 (UTC),2017-01-01 05:22 (UTC),992,7048.0,Front St / Yonge St (Hockey Hall of Fame),7089.0,Church St / Wood St,Member,NaN,NaN
3,712444,2017-01-01 05:09 (UTC),2017-01-01 05:26 (UTC),1005,7177.0,East Liberty St / Pirandello St,7202.0,Queen St W / York St (City Hall),Member,NaN,NaN
4,712445,2017-01-01 05:14 (UTC),2017-01-01 05:25 (UTC),645,7203.0,Bathurst St / Queens Quay W,7010.0,King St W / Spadina Ave,Member,NaN,NaN


After concatenation of different CSV files, the next step is to convert `Start_time` and `End_time` into `datetime` format. To do this, first we should take a look at these two columns. As it can be seen, these two columns contains string values such as `(UTC)` and `(ETS)`. So, we should remove this string from these two columns, as follow:
- add a new column containing the time zone of `Strat_Time` column. we use this column later to localize these two columns since they are localized based on different time zones
- removing values `(UTC)` and `(ETS)` from both colums

In [14]:
bikerides_df['Time_Zone'] = bikerides_df.apply(
    lambda row: row["Start_Time"].split("(")[1].split(")")[0] if ("(" in row["Start_Time"]) else False, axis=1)     
bikerides_df['Start_Time'] = bikerides_df.apply(lambda row: row["Start_Time"].split("(")[0], axis=1)
bikerides_df['End_Time']   = bikerides_df.apply(
    lambda row: row["End_Time"].split("(")[0] if type(row["End_Time"]) == str else "delete", axis=1)

In [15]:
bikerides_df.head()

,ID,Start_Time,End_Time,Duration,Start_Station_id,Start_Station_name,End_Station_id,End_Station_name,User_Type,Subscription Id,Bike Id,Time_Zone
0,712441,2017-01-01 05:03,2017-01-01 05:08,274,7006.0,Bay St / College St (East Side),7021.0,Bay St / Albert St,Member,NaN,NaN,UTC
1,712442,2017-01-01 05:03,2017-01-01 05:12,538,7046.0,Niagara St / Richmond St W,7147.0,King St W / Fraser Ave,Member,NaN,NaN,UTC
2,712443,2017-01-01 05:05,2017-01-01 05:22,992,7048.0,Front St / Yonge St (Hockey Hall of Fame),7089.0,Church St / Wood St,Member,NaN,NaN,UTC
3,712444,2017-01-01 05:09,2017-01-01 05:26,1005,7177.0,East Liberty St / Pirandello St,7202.0,Queen St W / York St (City Hall),Member,NaN,NaN,UTC
4,712445,2017-01-01 05:14,2017-01-01 05:25,645,7203.0,Bathurst St / Queens Quay W,7010.0,King St W / Spadina Ave,Member,NaN,NaN,UTC


There are some error in converting `End_Time` column into a datetime format which arises from that some values in this column are either nan or null. So, we should find these values and drop their records from the dataset

In [16]:
bikerides_df = bikerides_df.drop(index = bikerides_df[bikerides_df["End_Time"] == "delete"].index[0])

After removing the string parts from these two columns, we simply convert them to `datetime` format using following line

In [17]:
bikerides_df['Start_Time'] = pd.to_datetime(bikerides_df['Start_Time'])
bikerides_df['End_Time'] = pd.to_datetime(bikerides_df['End_Time'])  

Now, `Start_time` and `End_time` are of the datetime form, but they strill contain two different time zones. so in order to make it possible to work with them, we should convert all time zone into `EST`. To do this, first we define a function, then apply the function on each column to have uniform timezones

In [18]:
def UTC_To_EST(x):
    """
    Converting datetime for UTC to EST
    """
    UTC_TO_EST = pd.Timestamp(x).tz_localize('UTC')
    return UTC_TO_EST.tz_convert('EST')


bikerides_df['Start_Time'] = bikerides_df.apply(
    lambda row: UTC_To_EST(row['Start_Time']) if row["Time_Zone"] == "UTC" else row['Start_Time'], axis=1)
bikerides_df['End_Time'] = bikerides_df.apply(
    lambda row: UTC_To_EST(row['End_Time']) if row["Time_Zone"] == "UTC" else row['End_Time'], axis=1)

In [19]:
bikerides_df.head()

,ID,Start_Time,End_Time,Duration,Start_Station_id,Start_Station_name,End_Station_id,End_Station_name,User_Type,Subscription Id,Bike Id,Time_Zone
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006.0,Bay St / College St (East Side),7021.0,Bay St / Albert St,Member,NaN,NaN,UTC
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046.0,Niagara St / Richmond St W,7147.0,King St W / Fraser Ave,Member,NaN,NaN,UTC
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048.0,Front St / Yonge St (Hockey Hall of Fame),7089.0,Church St / Wood St,Member,NaN,NaN,UTC
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177.0,East Liberty St / Pirandello St,7202.0,Queen St W / York St (City Hall),Member,NaN,NaN,UTC
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203.0,Bathurst St / Queens Quay W,7010.0,King St W / Spadina Ave,Member,NaN,NaN,UTC


In this step, we try to localize all datetime to EST since, in the previous step, we only convert `UTC` ones to `EST`

In [20]:
#Localizing Timestamps 
bikerides_df["Start_Time"] = bikerides_df.apply(
    lambda row: row["Start_Time"] if (
        str(row["Start_Time"].tzinfo) == "EST") else row["Start_Time"].tz_localize(tz ='EST'), axis = 1)
bikerides_df["End_Time"] = bikerides_df.apply(
    lambda row: row["End_Time"] if (
        str(row["End_Time"].tzinfo) == "EST") else row["End_Time"].tz_localize(tz ='EST'), axis = 1) 

In [21]:
bikerides_df.head()

,ID,Start_Time,End_Time,Duration,Start_Station_id,Start_Station_name,End_Station_id,End_Station_name,User_Type,Subscription Id,Bike Id,Time_Zone
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006.0,Bay St / College St (East Side),7021.0,Bay St / Albert St,Member,NaN,NaN,UTC
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046.0,Niagara St / Richmond St W,7147.0,King St W / Fraser Ave,Member,NaN,NaN,UTC
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048.0,Front St / Yonge St (Hockey Hall of Fame),7089.0,Church St / Wood St,Member,NaN,NaN,UTC
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177.0,East Liberty St / Pirandello St,7202.0,Queen St W / York St (City Hall),Member,NaN,NaN,UTC
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203.0,Bathurst St / Queens Quay W,7010.0,King St W / Spadina Ave,Member,NaN,NaN,UTC


Let's take a look at the `Duration`. We should check that if there is any missing.

In [22]:
bikerides_df["Duration"].isnull().sum()

0

We can see that there is no missing values for `Duration`. Lets check that all values are of integer type or not.

Lets take a look at `User_Type`.

we can see that we have different terms for each of annual and casual members. As it can be seen we have two keywords for Annual Members and two for casual members

Lets make it uniform

In [23]:
bikerides_df["User_Type"].unique()

array(['Member', 'Casual', 'Annual Member', 'Casual Member'], dtype=object)

In [24]:
bikerides_df["User_Type"] = bikerides_df["User_Type"].apply(lambda row: "Casual" if ((row == "Casual") | (row == "Casual Member")) else "Annual")

Let's check the unique values of `User_Type` columns. 

There we go

In [25]:
bikerides_df["User_Type"].unique()

array(['Annual', 'Casual'], dtype=object)

We don't need `Time_zone` column anymore. so let's drop it.

In [26]:
bikerides_df.drop("Time_Zone", axis = 1, inplace = True)

Let's take a another look at the dataframe

In [27]:
bikerides_df.head()

,ID,Start_Time,End_Time,Duration,Start_Station_id,Start_Station_name,End_Station_id,End_Station_name,User_Type,Subscription Id,Bike Id
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006.0,Bay St / College St (East Side),7021.0,Bay St / Albert St,Annual,NaN,NaN
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046.0,Niagara St / Richmond St W,7147.0,King St W / Fraser Ave,Annual,NaN,NaN
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048.0,Front St / Yonge St (Hockey Hall of Fame),7089.0,Church St / Wood St,Annual,NaN,NaN
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177.0,East Liberty St / Pirandello St,7202.0,Queen St W / York St (City Hall),Annual,NaN,NaN
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203.0,Bathurst St / Queens Quay W,7010.0,King St W / Spadina Ave,Annual,NaN,NaN


we should check `Subscription Id` and `Bike Id` columns. 

In [28]:
bikerides_df["Subscription Id"].describe().dtype

dtype('float64')

In [29]:
bikerides_df["Bike Id"].describe().dtype

dtype('float64')

As it can be seen, these two columns are of the float type. let's convert them into an integer format

In [30]:
bikerides_df["Subscription Id"] = bikerides_df["Subscription Id"].astype('Int64')
bikerides_df["Bike Id"] = bikerides_df["Bike Id"].astype('Int64')
bikerides_df.head()

,ID,Start_Time,End_Time,Duration,Start_Station_id,Start_Station_name,End_Station_id,End_Station_name,User_Type,Subscription Id,Bike Id
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006.0,Bay St / College St (East Side),7021.0,Bay St / Albert St,Annual,<NA>,<NA>
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046.0,Niagara St / Richmond St W,7147.0,King St W / Fraser Ave,Annual,<NA>,<NA>
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048.0,Front St / Yonge St (Hockey Hall of Fame),7089.0,Church St / Wood St,Annual,<NA>,<NA>
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177.0,East Liberty St / Pirandello St,7202.0,Queen St W / York St (City Hall),Annual,<NA>,<NA>
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203.0,Bathurst St / Queens Quay W,7010.0,King St W / Spadina Ave,Annual,<NA>,<NA>


let's take a look at `Start_Station_id`, `Start_Station_name`, `End_Station_id`, and `End_Station_name`. There are two problems in these four columns.
- First, out of 12 Million records, more than one million records have no either `Start_Station_id` or `End_Station_id` or both.
- Second, There are issues with the consistency of station names in both `Start_Station_name` and `End_Station_name`. 

To solve these problems, firstly, we try to convert `Start_Station_id` and `End_Station_id` from `float` type to `integer` format. Then we will try to fill missing cells of `Start_Station_id` and `End_Station_id` with correct values. 
Let's convert these columns to integer.


In [31]:
bikerides_df["Start_Station_id"] = bikerides_df["Start_Station_id"].astype('Int64')
bikerides_df["End_Station_id"] = bikerides_df["End_Station_id"].astype('Int64')
bikerides_df.head()


,ID,Start_Time,End_Time,Duration,Start_Station_id,Start_Station_name,End_Station_id,End_Station_name,User_Type,Subscription Id,Bike Id
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006,Bay St / College St (East Side),7021,Bay St / Albert St,Annual,<NA>,<NA>
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046,Niagara St / Richmond St W,7147,King St W / Fraser Ave,Annual,<NA>,<NA>
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048,Front St / Yonge St (Hockey Hall of Fame),7089,Church St / Wood St,Annual,<NA>,<NA>
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177,East Liberty St / Pirandello St,7202,Queen St W / York St (City Hall),Annual,<NA>,<NA>
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203,Bathurst St / Queens Quay W,7010,King St W / Spadina Ave,Annual,<NA>,<NA>


Afterward we will fill missing values. A good way to fill this missing values is to use the `station names` columns. Firstly, we will create a dictionary of station names and their ids, and then we will apply this dictionary to the missing values.

In [32]:
#Creating two temporary dataframe for extracting unique station names and satation ids
df_temp1_station = bikerides_df[["Start_Station_name","Start_Station_id","Duration"]].groupby([
    "Start_Station_name","Start_Station_id"]).count().sort_values(by="Start_Station_id").reset_index().drop(
    "Duration", axis =1).rename(columns={"Start_Station_name": "name", "Start_Station_id": "id"})

df_temp2_station =bikerides_df[["End_Station_name","End_Station_id","Duration"]].groupby([
    "End_Station_name","End_Station_id"]).count().sort_values(by="End_Station_id").reset_index().drop(
    "Duration", axis =1).rename(columns={"End_Station_name": "name", "End_Station_id": "id"})

#Concatenation of two temporary dataframe to have a dataframe of unique names and ids
df_temp_station = pd.concat([df_temp1_station, df_temp2_station], ignore_index=True).sort_values(
    by="id").drop_duplicates(subset=["name","id"], keep='first')

# Converting dataframe to a dictionary
dicts_temp = df_temp_station.groupby('id')['name'].agg(list).to_dict()
dicts_temp_inv = df_temp_station.groupby('name')['id'].agg(list).to_dict()

dicts_temp[8000] = ["Michael Sweet Ave / St. Patrick St"]
dicts_temp_inv["Michael Sweet Ave / St. Patrick St"] = [8000]

In [34]:
#finding the indices of records with either no Start_Staion_id or End_Staion_id
temp_lst_no_start_id = bikerides_df[bikerides_df["Start_Station_id"].isnull() == True].index.tolist()
temp_lst_no_end_id = bikerides_df[bikerides_df["End_Station_id"].isnull() == True].index.tolist()

# temp_lst_no_start_id = temp_lst_no_start_id[0:4]

In [36]:
#filling missing values in Start_station_id column
for items in temp_lst_no_start_id:
    if bikerides_df["Start_Station_name"].iloc[items] in dicts_temp_inv.keys():

        bikerides_df["Start_Station_id"].iloc[items] = dicts_temp_inv.get(
            bikerides_df["Start_Station_name"].iloc[items])[0]
    else:

        bikerides_df["Start_Station_id"].iloc[items] = 1000
        

In [37]:
#filling missing values in End_station_id column
for items in temp_lst_no_end_id:
    if bikerides_df["End_Station_name"].iloc[items] in dicts_temp_inv.keys():

        bikerides_df["End_Station_id"].iloc[items] = dicts_temp_inv.get(
            bikerides_df["End_Station_name"].iloc[items])[0]
    else:

        bikerides_df["End_Station_id"].iloc[items] = 2000

Lets take a look at the dataframe 

In [41]:
bikerides_df.head()

,ID,Start_Time,End_Time,Duration,Start_Station_id,Start_Station_name,End_Station_id,End_Station_name,User_Type,Subscription Id,Bike Id
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006,Bay St / College St (East Side),7021,Bay St / Albert St,Annual,<NA>,<NA>
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046,Niagara St / Richmond St W,7147,King St W / Fraser Ave,Annual,<NA>,<NA>
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048,Front St / Yonge St (Hockey Hall of Fame),7089,Church St / Wood St,Annual,<NA>,<NA>
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177,East Liberty St / Pirandello St,7202,Queen St W / York St (City Hall),Annual,<NA>,<NA>
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203,Bathurst St / Queens Quay W,7010,King St W / Spadina Ave,Annual,<NA>,<NA>


In this stage, we will drop records which we could not find a matching id for their stations. Out of more than 12 Million records, we drop nearly 36000 records which is about 0.3% of data.

In [44]:
bikerides_df = bikerides_df.drop(bikerides_df[
    (bikerides_df["Start_Station_id"] == 1000) | (bikerides_df["End_Station_id"] == 2000) ].index)
bikerides_df.shape[0]

12163646

After cleaning dataframe, it is time to merge bikerides_df with weather dataframe, but since bikerides is of a minute timeseries resoultion and df_weather is of an hour scale, we should add a new column in bikerides_df which has an hour resolution. Let's add new column as `merge_time`

In [46]:
# Write your code here
bikerides_df['merge_time'] = bikerides_df.apply(lambda row: row["Start_Time"].round(freq='H') , axis=1)


Now, we can merge these two datasets

In [48]:
merged_df = bikerides_df.merge(df_weather, how ='left', left_on='merge_time', right_on='Date/Time')

# View DataFrame
merged_df.head()

,ID,Start_Time,End_Time,Duration,Start_Station_id,Start_Station_name,End_Station_id,End_Station_name,User_Type,Subscription Id,...,Dew Point Temp (°C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Stn Press Flag,Hmdx,Wind Chill,Weather
0,712441,2017-01-01 00:03:00-05:00,2017-01-01 00:08:00-05:00,274,7006,Bay St / College St (East Side),7021,Bay St / Albert St,Annual,<NA>,...,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN,NaN
1,712442,2017-01-01 00:03:00-05:00,2017-01-01 00:12:00-05:00,538,7046,Niagara St / Richmond St W,7147,King St W / Fraser Ave,Annual,<NA>,...,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN,NaN
2,712443,2017-01-01 00:05:00-05:00,2017-01-01 00:22:00-05:00,992,7048,Front St / Yonge St (Hockey Hall of Fame),7089,Church St / Wood St,Annual,<NA>,...,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN,NaN
3,712444,2017-01-01 00:09:00-05:00,2017-01-01 00:26:00-05:00,1005,7177,East Liberty St / Pirandello St,7202,Queen St W / York St (City Hall),Annual,<NA>,...,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN,NaN
4,712445,2017-01-01 00:14:00-05:00,2017-01-01 00:25:00-05:00,645,7203,Bathurst St / Queens Quay W,7010,King St W / Spadina Ave,Annual,<NA>,...,-3.6,69.0,26.0,39.0,16.1,99.81,NaN,NaN,NaN,NaN


In [52]:
merged_df.to_csv('merged_df_Mar27th_19.csv', index=False )
